In [1]:
# Instalação
!pip install yfinance==0.2.41
!pip install crewai==0.28.8
!pip install 'crewai[tools]'
!pip install langchain==0.1.20
!pip install langchain-openai==0.1.7
!pip install langchain-community==0.0.38
!pip install duckduckgo-search==5.3.0

In [25]:
import json
import os
from datetime import datetime

import yfinance as yf

from crewai import Agent, Task, Crew, Process
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

from IPython.display import Markdown

In [3]:
def fetch_stock_price(ticket):
    stock = yf.download(ticket, start="2023-08-08", end="2024-08-24")
    return stock

yahoo_finance_tool = Tool(
    name = "Yahoo Finance Tool",
    description = "Fetches stock prices for {ticket} from the last year about a specific stock from Yahoo Finance API",
    func = lambda ticket: fetch_stock_price(ticket)
)

In [4]:
response = yahoo_finance_tool.run("AAPL")

[*********************100%%**********************]  1 of 1 completed


In [5]:
print(response)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  179.690002  180.270004  177.580002  179.800003  178.643387   
2023-08-09  180.869995  180.929993  177.009995  178.190002  177.043762   
2023-08-10  179.479996  180.750000  177.600006  177.970001  176.825150   
2023-08-11  177.320007  178.619995  176.550003  177.789993  176.884857   
2023-08-14  177.970001  179.690002  177.309998  179.460007  178.546371   
...                ...         ...         ...         ...         ...   
2024-08-15  224.600006  225.350006  222.759995  224.720001  224.720001   
2024-08-16  223.919998  226.830002  223.649994  226.050003  226.050003   
2024-08-19  225.720001  225.990005  223.039993  225.889999  225.889999   
2024-08-20  225.770004  227.169998  225.449997  226.509995  226.509995   
2024-08-21  226.520004  227.979996  225.050003  226.399994  226.399994   

              Volume  
Date          

In [6]:
# Importing OpenAI LLM
os.environ['OPENAI_API_KEY'] = 
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [7]:
stockPriceAnalyst = Agent(
    role="Senior stock price Analyst",
    goal="Find the {ticket} stock price and analyses the price of an specific stack",
    backstory="""You're highly experienced in analyzing the price of an specific stock
    and make predicitions about its future price""",
    verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
    allow_delegation = False,
    tools=[yahoo_finance_tool]
)

In [8]:
getStockPrice = Task(
    description = "Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
    expected_output = """Specify the current trend stock price - up, down or sideways.
    eg. stock = 'APPL, price UP'
    """,
    agent = stockPriceAnalyst
)

In [9]:
# Importing Search tool
search_tool = DuckDuckGoSearchResults(backend="news", num_results=10)

In [10]:
newsAnalyst = Agent(
    role="Stock News Analyst",
    goal="""Create a short summary of the market news related to the stock {ticket} company. Specify the current trend - up, down or sideways 
    with the news context. For each request stock asset, specify a number between 0 and 100, where 0 is extreme fear and 100 is extreme greed
    """,
    backstory="""You're highly experienced in analyzing the market trends and news and have tracked assets for more than 10 years.

    You're also master level analyst in the tradicional markets and have deep understanding of human psychology.

    You understand news, theirs titles and information, but look at those with a health dose of skepticist.
    You consider also the source of the news articles.
    """,
    verbose=True,
    llm=llm,
    max_iter=10,
    memory=True,
    allow_delegation = False,
    tools=[search_tool]
)

In [16]:
getNews = Task(
    description = """Take the stock and always include BTC to it (if not request)
    Use the search tool to search earch on individually

    The current date is {datetime},

    Compose the results into a helpfull report
    """,
    expected_output = """A summary of the overall market and one sentence summary for each request asset.
    Include a fear/greed score for each asset based on the news. Use format:
    <STOCK ASSET>
    <SUMMARY BASED ON NEWS>
    <TREND PREDICTION>
    <FEAR/GREED SCORE>
    """,
    agent = newsAnalyst 
)

In [17]:
stockAnalystWrite = Agent(
    role="Senior Stock Analyst writer",
    goal="""Analyze the trends price and write an insightfull compelling and informative 3 paragraph long newsletter based on the stock report and price trend.
    """,
    backstory="""You're widely accepted as the best stock analyst in the market. You understand complex concepts and create compelling stories and narratives that resonate with wider audiences.

    You understand macro factors and combine multiple theories - eg. cycle theory and fundamental analyses. 
    You're able to hold multple opinios when analyzing anything.
    """,
    verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
    allow_delegation = True,
)

In [18]:
writeAnalyses = Task(
    description = """Use the stock price trend and the news report to create and analyses and write the newsletter about the {ticket} company
    that is brief and highlights the most important points.
    Focus on the stock price trend, news and fear/greed score. What are the near future considerations?
    Include the previous analyses of stock trend and news summary. 
    """,
    expected_output = """An eloquent 3 paragraphs newsletter formated as markdown in an easy readable manner. It should contain:

    - 3 bullets executive summary
    - Introduction - set the overall picture and spike up the interest
    - Main part provides the meat of the analysis including the news summary and fear/greed scores
    - summary - key facts and concrete future trend prediction - up, down or sideways
    """,
    agent = stockAnalystWrite,
    context = [getStockPrice, getNews]
)

In [19]:
# Create crew
crew = Crew(
    agents = [stockPriceAnalyst, newsAnalyst, stockAnalystWrite],
    tasks = [getStockPrice, getNews, writeAnalyses],
    verbose = True,
    process = Process.hierarchical,
    full_output = True,
    share_crew=False,
    manager_llm=llm,
    max_iter=15
)


2024-08-22 00:18:42,174 - 133341617513536 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [27]:
results = crew.kickoff(inputs={'ticket': 'AAPL', 'datetime': datetime.now().strftime("%Y-%m-%d %H:%M:%S")})

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Analyze the stock AAPL price history and create a trend analyses of up, down or sideways


> Entering new CrewAgentExecutor chain...
I need to gather information on the stock AAPL price history and create a trend analysis to determine if the current trend is up, down, or sideways.

Action: Delegate work to co-worker
Action Input: {"coworker": "Senior stock price Analyst", "task": "Analyze AAPL price history and determine trend analysis", "context": "I need you to analyze the historical stock price of AAPL and determine if the current trend is up, down, or sideways. Please provide a detailed analysis based on the data available."}

> Entering new CrewAgentExecutor chain...
I need to fetch the historical stock prices of AAPL from the last year in order to analyze the trend.

Action: Yahoo Finance Tool
Action Input: {"ticket": "AAPL"}

[*********************100%%**********************]  1 of 1 completed


 

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  179.690002  180.270004  177.580002  179.800003  178.643402   
2023-08-09  180.869995  180.929993  177.009995  178.190002  177.043747   
2023-08-10  179.479996  180.750000  177.600006  177.970001  176.825180   
2023-08-11  177.320007  178.619995  176.550003  177.789993  176.884857   
2023-08-14  177.970001  179.690002  177.309998  179.460007  178.546356   
...                ...         ...         ...         ...         ...   
2024-08-15  224.600006  225.350006  222.759995  224.720001  224.720001   
2024-08-16  223.919998  226.830002  223.649994  226.050003  226.050003   
2024-08-19  225.720001  225.990005  223.039993  225.889999  225.889999   
2024-08-20  225.770004  227.169998  225.449997  226.509995  226.509995   
2024-08-21  226.520004  227.979996  225.050003  226.399994  226.399994   

              Volume  
Date       

In [28]:
list(results.keys())

['final_output', 'tasks_outputs']

In [29]:
results["final_output"]

"# Market Analysis Newsletter\n\n## Executive Summary\n- AAPL stock price trend is on the rise, indicating positive movement in the technology sector.\n- Bitcoin experiences volatility but maintains an overall uptrend, driven by institutional adoption and regulatory developments.\n- Investors should conduct thorough research before making investment decisions amidst dynamic market conditions.\n\n## Introduction\nIn today's market report, we have observed an upward trend in the AAPL stock price, signaling positive movement in the technology sector. This could potentially reflect broader market sentiment and economic conditions, hinting at a bullish outlook. On the other hand, Bitcoin has been experiencing volatility but maintaining an overall uptrend, driven by factors such as institutional adoption and regulatory developments. This resilience in Bitcoin amidst macroeconomic uncertainties highlights the growing interest in alternative assets like cryptocurrencies.\n\n## Main Analysis\nT

In [30]:
len(results["tasks_outputs"])

3

In [31]:
Markdown(results["final_output"])

# Market Analysis Newsletter

## Executive Summary
- AAPL stock price trend is on the rise, indicating positive movement in the technology sector.
- Bitcoin experiences volatility but maintains an overall uptrend, driven by institutional adoption and regulatory developments.
- Investors should conduct thorough research before making investment decisions amidst dynamic market conditions.

## Introduction
In today's market report, we have observed an upward trend in the AAPL stock price, signaling positive movement in the technology sector. This could potentially reflect broader market sentiment and economic conditions, hinting at a bullish outlook. On the other hand, Bitcoin has been experiencing volatility but maintaining an overall uptrend, driven by factors such as institutional adoption and regulatory developments. This resilience in Bitcoin amidst macroeconomic uncertainties highlights the growing interest in alternative assets like cryptocurrencies.

## Main Analysis
The current trends suggest bullish momentum for AAPL and Bitcoin, with AAPL's upward trajectory aligning with a positive outlook for the technology sector. The volatility in Bitcoin's price is counterbalanced by its overall uptrend, driven by institutional adoption and regulatory developments. As investors navigate through these dynamic market conditions, conducting thorough research and considering all factors before making investment decisions is crucial.

## Summary
In conclusion, the market remains dynamic and subject to various external factors, but the current trends indicate a positive outlook for both AAPL and Bitcoin. Investors are advised to stay informed, conduct thorough research, and consider all factors before making any investment decisions. Stay tuned for more insights and analysis in our upcoming reports.

Best regards,
[Your Name] 
Senior Stock Analyst